In [1]:
import numpy as np
import pandas as pd
import os,sys
import cv2
import pickle

In [2]:
# initialize the list of points for the rectangle bbox,
# the temporaray endpoint of the drawing rectangle
# the list of all bounding boxes of selected rois
# and boolean indicating wether drawing of mouse
# is performed or not
rect_endpoint_tmp = []
rect_bbox = []

drawing = False

def check_boxes(img_clean,bbox_list):
    def draw_all_boxes():
        img = img_clean.copy()
        
        for b in bbox_list:
            cv2.rectangle(img, (b[0],b[1]),(b[2],b[3]), color=(0, 255, 0),thickness=1)
        cv2.imshow('image', img)
            
        
    # mouse callback function
    def draw_rect_roi(event, x, y, flags, param):
            # grab references to the global variables
            global rect_bbox, rect_endpoint_tmp, drawing
    
            # if the left mouse button was clicked, record the starting
            # (x, y) coordinates and indicate that drawing is being
            # performed. set rect_endpoint_tmp empty list.
            if event == cv2.EVENT_LBUTTONDOWN:
                rect_endpoint_tmp = []
                rect_bbox = [(x, y)]
                drawing = True
    
            # check to see if the left mouse button was released
            elif event == cv2.EVENT_LBUTTONUP:
                # record the ending (x, y) coordinates and indicate that
                # drawing operation is finished
                rect_bbox.append((x, y))
                drawing = False
    
                # draw a rectangle around the region of interest
                p_1, p_2 = rect_bbox
                
                # for bbox find upper left and bottom right points
                p_1x, p_1y = p_1
                p_2x, p_2y = p_2
    
                lx = min(p_1x, p_2x)
                ty = min(p_1y, p_2y)
                rx = max(p_1x, p_2x)
                by = max(p_1y, p_2y)
    
                # add bbox to list if both points are different
                if (lx, ty) != (rx, by):
                    if abs(lx-rx)>5:
                        if abs(ty-by)>5:
                            bbox = [lx, ty, rx, by]
                            bbox_list.append(bbox)
    
            # if mouse is drawing set tmp rectangle endpoint to (x,y)
            elif event == cv2.EVENT_MOUSEMOVE and drawing:
                rect_endpoint_tmp = [(x, y)]
            elif event == cv2.EVENT_LBUTTONDBLCLK:
                npbx=np.asarray(bbox_list)
                selected_box = ((x>npbx[:,0]) & (y>npbx[:,1]) & (x<npbx[:,2]) & (y<npbx[:,3]))
                if np.sum(selected_box)==1:
                    bbox_list.remove(npbx[selected_box].tolist()[0])
                if np.sum(selected_box)>1:
                    potentials = npbx[selected_box]
                    areas = (potentials[:,2]-potentials[:,0])*(potentials[:,3]-potentials[:,1])
                    bbox_list.remove(potentials[np.argmin(areas)].tolist())
                draw_all_boxes()

    cv2.namedWindow('image',cv2.WINDOW_GUI_NORMAL )
    cv2.resizeWindow('image', 900,900)
    cv2.setMouseCallback('image', draw_rect_roi)
    draw_all_boxes()
    # keep looping until the 'c' key is pressed
    stop = False
    while True:
        # display the image and wait for a keypress
        if not drawing:
            draw_all_boxes()
            #cv2.imshow('image', img)
        elif drawing and rect_endpoint_tmp:
            rect_cpy = img_clean.copy()
            start_point = rect_bbox[0]
            end_point_tmp = rect_endpoint_tmp[0]
            cv2.rectangle(rect_cpy, start_point, end_point_tmp,(0,255,0),1)
            cv2.imshow('image', rect_cpy)
    
        key = cv2.waitKey(1) #& 0xFF
        # if the 'c' key is pressed, break from the loop
        if key == ord('c'):
            break
        if key == ord('q'):
            stop=True
            break
    # close all open windows
    cv2.destroyAllWindows()
    #cv2.waitKey(1)
    return stop

In [6]:

im_dir = '/home/ctorney/euclid/workspace/uavTracker/train/horse_images/'

with open (im_dir + 'annotations-trained.pickle', 'rb') as fp:
    all_imgs = pickle.load(fp)



from_scratch=False
if from_scratch:
    new_imgs = []
else:
    with open (im_dir + 'annotations-checked.pickle', 'rb') as fp:
        new_imgs = pickle.load(fp)

for i in range(len(all_imgs)):
    basename = os.path.basename(all_imgs[i]['filename'])
    if not from_scratch:
        if any(d['filename'] == basename for d in new_imgs):
            continue
    img_data = {'object':[]}
    img_data['filename'] = basename
    img_data['width'] = all_imgs[i]['width']
    img_data['height'] = all_imgs[i]['height']
    
    print(img_data['filename'] + ", " + str(i) + ' of ' + str(len(all_imgs)))
    boxes=[]
    for obj in all_imgs[i]['object']:
        boxes.append([obj['xmin'],obj['ymin'],obj['xmax'],obj['ymax']])

    #do box processing
    img = cv2.imread(im_dir + basename)
    if check_boxes(img,boxes):
        break
    for b in boxes:
        obj = {}
        if ((b[2]-b[0])*(b[3]-b[1]))<10:
            continue
        obj['name'] = 'aoi'
        obj['xmin'] = int(b[0])
        obj['ymin'] = int(b[1])
        obj['xmax'] = int(b[2])
        obj['ymax'] = int(b[3])
        img_data['object'] += [obj]

    new_imgs += [img_data]

#print(all_imgs)
with open(im_dir + 'annotations-checked.pickle', 'wb') as handle:
   pickle.dump(new_imgs, handle)

TR_180601-4-8504_half-0.png, 210 of 2000
TR_180601-4-8504_half-1.png, 211 of 2000
TR_170614-4-2-5178_half-0.png, 212 of 2000
TR_170614-4-2-5178_half-1.png, 213 of 2000
TR_180530-13-9714_half-0.png, 214 of 2000
TR_180530-13-9714_half-1.png, 215 of 2000
TR_170606-2-3515_half-0.png, 216 of 2000
TR_170606-2-3515_half-1.png, 217 of 2000
TR_DJI_0007-8744_half-0.png, 218 of 2000
TR_DJI_0007-8744_half-1.png, 219 of 2000
TR_180615-14-823_half-0.png, 220 of 2000
TR_180615-14-823_half-1.png, 221 of 2000
TR_180528-9-2364_half-0.png, 222 of 2000
TR_180528-9-2364_half-1.png, 223 of 2000
TR_180615-18-8853_half-0.png, 224 of 2000
TR_180615-18-8853_half-1.png, 225 of 2000
TR_180517-9-15201_half-0.png, 226 of 2000
TR_180517-9-15201_half-1.png, 227 of 2000
TR_180616-5-5013_half-0.png, 228 of 2000
TR_180616-5-5013_half-1.png, 229 of 2000
TR_170608-5-2201_half-0.png, 230 of 2000
TR_170608-5-2201_half-1.png, 231 of 2000
TR_180521-13-3883_half-0.png, 232 of 2000
TR_180521-13-3883_half-1.png, 233 of 2000
TR_1